In [2]:
import os, time, math, random,re,glob
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
# import torch, torch_directml as dml
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel,get_linear_schedule_with_warmup
from carbontracker.tracker import CarbonTracker
from carbontracker import parser as ct_parser
import matplotlib.pyplot as plt

C:\Users\Dreamcore\anaconda3\envs\fyp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def find_ct_output_log(ct_dir, prefix:str | None = None, timeout_s: float = 3.0):
    search_roots = [os.path.abspath(ct_dir), os.path.abspath(os.getcwd())]
    deadline = time.time() + timeout_s
    best = None
    while time.time() <= deadline:
        candidates = []
        for root in search_roots:
            if os.path.isdir(root):
                candidates.extend(glob.glob(os.path.join(root, "**", "*carbontracker_output.log"), recursive=True))
        candidates = sorted(set(candidates), key=os.path.getmtime, reverse=True)
        if prefix:
            pref = [p for p in candidates if prefix in os.path.basename(p)]
            if pref:
                return pref[0]
        if candidates:
            return candidates[0]
        time.sleep(0.25)
    return best

def parse_carbontracker(ct_dir, prefix: str | None = None):
    output_log = None
    # Try official helper
    try:
        from carbontracker import parser as ct_parser
        output_log, _ = ct_parser.get_most_recent_logs(log_dir=ct_dir)
    except Exception:
        pass
    if not output_log or not os.path.exists(output_log):
        output_log = find_ct_output_log(ct_dir, prefix=prefix)
    if not output_log or not os.path.exists(output_log):
        return None, None, None, None

    with open(output_log, "r", encoding="utf-8", errors="ignore") as f:
        out_txt = f.read()
    try:
        from carbontracker import parser as ct_parser
        actual, pred = ct_parser.get_consumption(out_txt)
        cons = actual or pred
        if cons:
            e = cons.get("energy (kWh)")
            c = cons.get("co2eq (g)")
            d = cons.get("duration (s)")
            return (
                float(e) if e is not None else None,
                float(c) if c is not None else None,
                float(d) if d is not None else None,
                output_log,
            )
    except Exception:
        pass

    #regex fallback
    mE = re.search(r"Energy:\s*([0-9.eE+-]+)\s*kWh", out_txt)
    mC = re.search(r"CO2eq:\s*([0-9.eE+-]+)\s*g", out_txt)
    mT = re.search(r"Time:\s*([0-9:]+)", out_txt)

    energy_kwh = float(mE.group(1)) if mE else None
    co2_g      = float(mC.group(1)) if mC else None
    duration_s = None
    if mT:
        hh, mm, ss = (int(x) for x in mT.group(1).split(":"))
        duration_s = 3600*hh + 60*mm + ss

    return energy_kwh, co2_g, duration_s, output_log


### 1. Data Loading

data loading for YELP (train dataset) and IMDB (test dataset)

In [6]:
#YELP

yelp_neg= pd.read_csv(r"C:\Users\Dreamcore\Downloads\yelp_2_and_below")
yelp_pos = pd.read_csv(r"C:\Users\Dreamcore\Downloads\yelp_4_and_above")

#IMDB (test)
imdb = pd.read_csv(r"C:\Users\Dreamcore\Downloads\archive\IMDB Dataset.csv")


setup of device

In [7]:
# device = dml.device(dml.default_device())
device = "cuda"

### 2. Preprocessing

adding '1' for positive entries, '0' for negative entries

In [8]:
yelp_neg['sentiment'] = 'negative'
yelp_pos['sentiment'] = 'positive'

keeping review(text) + sentiment

In [9]:
columns = ['text','sentiment']
yelp = pd.concat([yelp_neg[columns], yelp_pos[columns]], ignore_index=True)

In [10]:
imdb = imdb.rename(columns={'review':'text'})

mapping positive to integer 1, and negative to integer 0

In [11]:
num = {'negative': 0, 'positive': 1}
for df in (yelp, imdb):
    #normalize column names if needed
    df['sentiment'] = df['sentiment'].str.lower().str.strip()
    df['label'] = df['sentiment'].map(num)

### 3. Training Setup

Train-validation split

In [12]:
SEED = 20
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
yelp_train, yelp_val = train_test_split(
    yelp, test_size=0.1, random_state=SEED, stratify=yelp['sentiment'])

text class

In [13]:
BATCH_SIZE = 16

class TextDataset(Dataset):
    def __init__(self, df):
        self.texts  = df['text'].astype(str).tolist()
        self.labels = df['label'].astype(int).tolist()
    def __len__(self): return len(self.texts)
    def __getitem__(self, i): return self.texts[i], self.labels[i]

def make_loader(df, tokenizer, shuffle):
    def collate_fn(batch):
        texts, labels = zip(*batch)
        enc = tokenizer(list(texts), truncation=True, padding=True,return_tensors='pt')
        enc['labels'] = torch.tensor(labels, dtype=torch.long)
        return enc
    ds = TextDataset(df)
    return DataLoader(ds, batch_size=BATCH_SIZE, shuffle=shuffle, collate_fn=collate_fn)

CNN head, shared among both BERT models

In [14]:
class TransformerCNN(nn.Module):
    def __init__(self, model_name, num_labels=2, cnn_filters=256, kernel_size=3, dropout=0.1):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(model_name)
        hidden = self.encoder.config.hidden_size
        self.conv = nn.Conv1d(in_channels=hidden, out_channels=cnn_filters,
                              kernel_size=kernel_size, padding=kernel_size//2)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(cnn_filters, num_labels)

    def forward(self, input_ids, attention_mask, labels=None):
        out = self.encoder(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        x = out.last_hidden_state
        x = x.transpose(1, 2)
        x = torch.relu(self.conv(x))
        x = torch.max(x, dim=2).values
        x = self.dropout(x)
        logits = self.classifier(x)
        loss = None
        if labels is not None:
            loss = nn.CrossEntropyLoss()(logits, labels)
        return {"loss": loss, "logits": logits}

training, test loops

In [15]:
def run_epoch(model, loader, optimizer=None, scheduler=None, training=True):
    model.train(training)
    losses, preds_all, labels_all = [], [], []

    for batch in loader:
        input_ids= batch['input_ids'].to(device)
        attention_mask= batch['attention_mask'].to(device)
        labels= batch['labels'].to(device).long()

        output= model(input_ids=input_ids,attention_mask=attention_mask,labels=labels)
        loss = output['loss']

        if training:
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            if scheduler is not None:
                scheduler.step()

        losses.append(loss.item())

        preds_np  = torch.argmax(output['logits'], dim=1).detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()
        preds_all.extend(preds_np)
        labels_all.extend(labels_np)

    acc = accuracy_score(labels_all, preds_all)
    f1  = f1_score(labels_all, preds_all, average='weighted')
    return float(np.mean(losses)), acc, f1

@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    losses, preds_all, labels_all = [], [], []
    for batch in loader:
        output = model(input_ids=batch['input_ids'].to(device),
                    attention_mask=batch['attention_mask'].to(device),
                    labels=batch['labels'].to(device))
        losses.append(output['loss'].item())
        preds = torch.argmax(output['logits'], dim=1).detach().cpu().numpy()
        labels = batch['labels'].detach().cpu().numpy()
        preds_all.extend(preds)
        labels_all.extend(labels)
    acc = accuracy_score(labels_all, preds_all)
    f1  = f1_score(labels_all, preds_all, average='weighted')
    class_rep = classification_report(labels_all, preds_all, target_names=['negative','positive'])
    return float(np.mean(losses)), acc, f1, class_rep

def train_and_eval(model_name, epochs=10, lr=1e-5, patience=3,min_delta=0.01,plot_dir="plots",tracker_dir="energy_logs"):
    os.makedirs(plot_dir, exist_ok=True)
    os.makedirs(tracker_dir,exist_ok=True)
    safe_name = re.sub(r"[^a-zA-Z0-9_.-]+", "_", model_name)

    tokenizer    = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    train_loader = make_loader(yelp_train, tokenizer, shuffle=True)
    val_loader   = make_loader(yelp_val,   tokenizer, shuffle=False)
    test_loader  = make_loader(imdb,tokenizer, shuffle=False)

    model = TransformerCNN(model_name).to(device)

    steps_per_epoch = math.ceil(len(yelp_train) / BATCH_SIZE)
    total_steps     = epochs * steps_per_epoch
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, int(0.05* total_steps),total_steps
    )
    
    tracker = CarbonTracker(epochs=epochs,log_dir=tracker_dir,log_file_prefix=safe_name)
    history = []
    tr_losses, va_losses = [], []
    tr_accs,   va_accs   = [], []

    best_val = float("inf")
    best_state = None
    epochs_no_improve = 0
    early_stopped = False

    t0 = time.time()
    for ep in range(1, epochs + 1):
        if tracker: tracker.epoch_start()

        tr_loss, tr_acc, tr_f1 = run_epoch(model, train_loader, optimizer, scheduler, training=True)
        va_loss, va_acc, va_f1 = run_epoch(model, val_loader,   optimizer=None, scheduler=None, training=False)

        if tracker: tracker.epoch_end()

        tr_losses.append(tr_loss); va_losses.append(va_loss)
        tr_accs.append(tr_acc);   va_accs.append(va_acc)

        history.append(dict(
            epoch=ep,
            train_loss=tr_loss, train_acc=tr_acc, train_f1=tr_f1,
            val_loss=va_loss,   val_acc=va_acc,   val_f1=va_f1
        ))

        print(f"[{model_name}] Ep {ep:02d}/{epochs} | "
              f"train {tr_loss:.4f}/{tr_acc:.4f}/{tr_f1:.4f} | "
              f"val {va_loss:.4f}/{va_acc:.4f}/{va_f1:.4f}")

        #early stopping
        if va_loss + min_delta < best_val:
            best_val = va_loss
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                early_stopped = True
                print(f"[EarlyStopping] No val_loss improvement for {patience} epoch(s). Stopping at epoch {ep}.")
                break

    if tracker:
        tracker.stop()
    train_time_sec = time.time() - t0

    #restore best state
    if best_state is not None:
        model.load_state_dict(best_state)

    #final
    test_loss, test_acc, test_f1, class_rep = evaluate(model, test_loader)
    print(f"\n[{model_name}] TEST | loss={test_loss:.4f} acc={test_acc:.4f} f1={test_f1:.4f}")
    print(class_rep)

    #plots
    history_df  = pd.DataFrame(history)
    epochs_axis = range(1, len(tr_losses) + 1)

    plt.figure(figsize=(7, 5))
    plt.plot(epochs_axis, tr_losses, label="train loss")
    plt.plot(epochs_axis, va_losses, label="val loss")
    plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.title(f"Loss Curves — {model_name}")
    plt.legend()
    plt.grid(True, alpha=0.3)
    loss_path = os.path.join(plot_dir, f"{safe_name}_loss.png")
    plt.savefig(loss_path, bbox_inches="tight"); plt.close()

    plt.figure(figsize=(7, 5))
    plt.plot(epochs_axis, tr_accs, label="train acc")
    plt.plot(epochs_axis, va_accs, label="val acc")
    plt.xlabel("Epoch"); plt.ylabel("Accuracy"); plt.title(f"Accuracy Curves — {model_name}")
    plt.legend()
    plt.grid(True, alpha=0.3)
    acc_path = os.path.join(plot_dir, f"{safe_name}_acc.png")
    plt.savefig(acc_path, bbox_inches="tight"); plt.close()

    #tracker
    energy_kwh = co2_g = duration_s = None
    ct_devices = None
    try:
        output_log, standard_log = ct_parser.get_most_recent_logs(log_dir=tracker_dir)  # CHANGE
        with open(output_log, "r", encoding="utf-8") as f:
            out_txt = f.read()                                                          # CHANGE
        with open(standard_log, "r", encoding="utf-8") as f:
            std_txt = f.read()                                                          # CHANGE
        actual, pred = ct_parser.get_consumption(out_txt)
        cons = actual or pred
        if cons is not None:                                                            # CHANGE
            energy_kwh = cons.get("energy (kWh)")
            co2_g      = cons.get("co2eq (g)")
            duration_s = cons.get("duration (s)")
        ct_devices = ct_parser.get_devices(std_txt)                                     # CHANGE
    except Exception as e:
        print("[CarbonTracker] parse warning:", e)                                      # CHANGE

    # summary (kept clean — omit CT fields entirely if you prefer)
    summary = {
        "model": model_name,
        "early_stopped": early_stopped,
        "epochs_ran": len(tr_losses),
        "time_s_train": round(train_time_sec, 2),
        "test_loss": round(test_loss, 4),
        "test_acc":  round(test_acc, 4),
        "test_f1":   round(test_f1, 4),
        "loss_plot": loss_path,
        "acc_plot":  acc_path,
        "energy_kwh": None if energy_kwh is None else round(float(energy_kwh), 4),
        "co2_g": None if co2_g is None else round(float(co2_g), 2),
        "duration_s": None if duration_s is None else int(float(duration_s)),
        "devices": ct_devices
    }
    return model, history_df, summary

### 4. Training Loop

In [ ]:
#BERT base
bert_base = "bert-base-uncased"
#BERT small
bert_small = "prajjwal1/bert-small"

model_bertsmall, hist_bertsmall_df, summary_bertsmall = train_and_eval(
    bert_small, epochs=3, lr=2e-5, patience=2, min_delta=0.0,
    plot_dir="plots", tracker_dir="carbon_logs"
)
print(summary_bertsmall)

In [17]:
model_bertbase, hist_bertbase_df, summary_bertbase = train_and_eval(
    bert_base, epochs=3, lr=2e-5, patience=2, min_delta=0.0,
    plot_dir="plots", tracker_dir="carbon_logs"
)

print(summary_bertbase)

C:\Users\Dreamcore\anaconda3\envs\fyp\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dreamcore\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CarbonTracker: INFO - Detected CPU: 12th Gen Intel(R) Core(TM) i5-12400F
CarbonTracker: WARNING - No matching TDP found for CPU: 12th Gen Intel(R) Core(TM) i5-12400F. Using average TDP of 35.61W at 50% utilization as fallback.
CarbonTracker: WARNING - No API keys provided. Skipping intensity provider initialization.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti. CPU with device(s) 12th Gen Intel(R) Core(TM) i5-12400F.
CarbonTracker: WARNING - No carbon intensity provider specified. Using average carbon intensity for SG: 498.74 gCO2eq/kWh.
CarbonTracker: 
Predicted consumption for 3 epoch(s):
	Time:	2:25:24
	Energy:	0.520018890368 kWh
	CO2eq:	259.355167816494 g
	This is equivalent to:
	2.428419174312 km travelled by car
[bert-base-uncased] Ep 01/3 | train 0.2311/0.9031/0.9031 | val 0.0901/0.9740/0.9740
[bert-base-uncased] Ep 02/3 | train 0.0713/0.9802/0.9802 | val 0.1191/0.9710/0.9710
CarbonTracker: Average carbon intensity during trai